In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

rms = 0.401862
fx = 488.943879
fy = 490.295543
cx = 323.123703
cy = 240.284288
k1 = 0.067409
k2 = -0.035474
p1 = -0.002333
p2 = -0.001423


def auto_canny_otsu(image):
    # 오츠의 방법을 사용하여 엣지 검출을 위한 최적의 임계값을 계산
    high_thresh, _ = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    low_thresh = 0.5 * high_thresh

    # Canny 엣지 검출 적용
    blur = cv2.GaussianBlur(image, (3, 3), 0)
    edged = cv2.Canny(blur, low_thresh, high_thresh)

    return edged



In [4]:
import cv2
import numpy as np
import glob

# 가정: 체스보드 패턴의 크기 및 촬영된 이미지가 준비되어 있음
chessboard_size = (9, 6)  # 체스보드의 코너점 수 (가로, 세로)
square_size = 2.9        # 체스보드 한 칸의 크기

# 3D 참조점 준비
objp = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
objp *= square_size

# 이미지에서 코너점 검출 및 캘리브레이션
objpoints = []  # 3D 참조점
imgpoints = []  # 2D 이미지점

image_path = glob.glob("./data/droid_calib/*.jpg")

images = [cv2.imread(image) for image in image_path]
# print(images)

for image in images:  # 'images'는 촬영된 이미지 리스트
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

    if ret:
        objpoints.append(objp)
        imgpoints.append(corners)

# print(objpoints)
# print(imgpoints)
        
# 카메라 캘리브레이션 수행
ret, K, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# 첫 번째 이미지에 대한 회전 벡터를 회전 행렬로 변환
R, _ = cv2.Rodrigues(rvecs[0])


In [9]:
K = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
# R = np.eye(3)
# t = np.zeros((3, 1))
# 회전행렬
R, _ = cv2.Rodrigues(rvecs[0])
t = tvecs[0]

# 첫 번째 카메라의 회전 벡터와 이동 벡터
rvec1 = rvecs[0]
tvec1 = tvecs[0]

# 회전 벡터를 회전 행렬로 변환
R1, _ = cv2.Rodrigues(rvec1)

# 첫 번째 카메라의 투영 행렬
P1 = K @ np.hstack((R1, tvec1))

# 두 번째 카메라의 회전 벡터와 이동 벡터 (예시)
# 실제로는 두 번째 카메라의 캘리브레이션 데이터가 필요합니다.
rvec2 = rvecs[1]  # 예시 값
tvec2 = tvecs[1]  # 예시 값

# 회전 벡터를 회전 행렬로 변환
R2, _ = cv2.Rodrigues(rvec2)

# 두 번째 카메라의 투영 행렬
P2 = K @ np.hstack((R2, tvec2))

cap = cv2.VideoCapture("http://192.168.0.9:4747/video")


# Lucas-Kanade 파라미터 설정
lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 
                           10, 0.03))

# 첫 번째 프레임에서 코너 검출
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, maxCorners=100, qualityLevel=0.01, minDistance=30)



while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 옵티컬 플로우 계산
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # 좋은 코너점 선택
    good_new = p1[st == 1]
    good_old = p0[st == 1]
    
    
    # 삼각측량 수행
    points_4d_hom = cv2.triangulatePoints(P1, P2, good_old.T, good_new.T)
    points_3d = points_4d_hom / points_4d_hom[3]
    points_3d = points_3d[:3].T
    
    
    # 동차 좌표에서 일반 좌표로 변환
    points_3d = points_4d_hom / points_4d_hom[3]
    points_3d = points_3d[:3].T
    
    
    # 중심 좌표(0, 0, 0)를 2D로 투영
    center_point_3d = np.array([[0.0, 0.0, 0.0]])
    center_point_2d = cv2.projectPoints(center_point_3d, R, t, K, None)[0]


    # 3D 점을 2D로 투영
    points_2d = cv2.projectPoints(points_3d, R, t, K, None)[0]
    
    edges = auto_canny_otsu(frame_gray)
    edges = cv2.cvtColor(edges ,cv2.COLOR_GRAY2BGR)
    
    # 2D 투영 점을 이미지에 표시하고, 해당 점의 3D 좌표를 텍스트로 추가
    for i, point in enumerate(points_2d):
        x, y = int(point[0][0]), int(point[0][1])
        cv2.circle(edges, (x, y), 5, (0, 255, 0), -1)

        # 3D 좌표 텍스트로 표시
        text = f"({points_3d[i][0]:.2f}, {points_3d[i][1]:.2f}, {points_3d[i][2]:.2f})"
        cv2.putText(edges, text, (x + 10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    x_center, y_center = int(center_point_2d[0][0][0]), int(center_point_2d[0][0][1])
    cv2.circle(edges, (x_center, y_center), 5, (0, 0, 255), -1)
    cv2.putText(edges, "Center (0, 0, 0)", (x_center + 10, y_center), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    cv2.imshow('frame', edges)

    # 다음 반복을 위해 현재 상태 업데이트
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

    if cv2.waitKey(20) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()